# Rebalance instances

### 1) Overall configurations 

In [1]:
import os.path
import pandas as pd
from pprint import pprint

# Root
root_path = os.getcwd().replace("\\","/")

# Instance settings
# What is the best simulation.rebalancing policy?
# instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/simulation.rebalancing/instance_settings_test_rebalancing.json"
# label = "reb_policy"

# Is it better to order the users per class?
instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/service_denial_ABC_req_time.json"
label = "deny_comparison_b1_x_b2_x_sortclasses"

#test_case = "standard_vs_enforce_BB"
#instance_settings_path = "C:/Users/LocalAdmin/IdeaProjects/slevels/src/main/resources/week/enforce_service_levels.json"
#data_dict_path = "../data/dictionary/request_track_data_dictionary.json"

# Data translation
dict_sl_status = {"MET":"Shortest", "UNMET":"Extended"}
dict_sq_class = {"A":"Business", "B":"Standard", "C":"Low-cost"}
dict_service = {"FLEET":"Company-owned", "FREELANCE":"Third-party"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["Business", "Standard", "Low-cost"], ordered=True)
category_fleet = pd.api.types.CategoricalDtype(categories=["Company-owned", "Third-party"], ordered=True)
category_status = pd.api.types.CategoricalDtype(categories=["Shortest", "Extended"], ordered=True)


### 2) Loading the instance data

In [2]:
import json

def load_json(path):
    """Read json file and return dictionary"""

    # Add .json to the end of file if needed
    if path.find(".json") < 0:
        path = path + ".json"

    # Read JSON file
    with open(path) as data_file:
        data_loaded = json.load(data_file)

    return data_loaded

instances_dic = load_json(instance_settings_path)

# Folder where results will be saved
result_folder = instances_dic["result_folder"]
request_log_folder = result_folder + "/request_track"

# Folder where instances are located
instances_folder = instances_dic["instances_folder"]

# File name aggregated data
instance_name = instances_dic["instance_name"]


print("########### INSTANCE SETTINGS ##################################################")
pprint(instances_dic)

########### INSTANCE SETTINGS ##################################################
{'distances_file': 'C:/Users/LocalAdmin/OneDrive/leap_forward/street_network_server/input_tripdata/data/manhattan-island-new-york-city-new-york-usa/dist/distance_matrix_m_manhattan-island-new-york-city-new-york-usa.csv',
 'instance_description': 'Whole week experiment, allow denying. Sort users '
                         'before inserting per class and request time.',
 'instance_name': 'WEEKDENYABCRT',
 'instances_folder': 'C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week_deny_ABCRT/',
 'labels': {'BA': 'batch_duration',
            'CD': 'contract_duration',
            'CS': 'customer_segmentation',
            'CT': 'clear_target_list_every_round',
            'ID': 'instance_description',
            'IF': 'initial_fleet',
            'IN': 'instance_name',
            'MC': 'max_capacity',
            'MO': 'allow_many_to_one',
            'MR': 'max_requests',
            'RE': 'rebalanc

### Get settings from instance name

E.g.:

* Input = `IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR`

* Output = 
{allow_many_to_one: True,
allow_service_deterioration: True,
allow_urgent_relocation: True,
allow_vehicle_hiring: True,
batch_duration: 30,
clear_target_list_every_round: True,
contract_duration: 3600,
customer_segmentation: AA,
initial_fleet: 1000,
max_capacity: 06,
max_requests: 1000,
reinsert_targets: True,
time_horizon: 86400}

In [3]:
def get_instance_settings(file_name):
    """ Read file name and return instance settings.
    E.g.:
     Input = IN-instanceName_BA-30_TH-86400_MR-1000_IF-1000_MC-06_CD-3600-SR-S1_CS-AA_SD_VH_MO_RT_CT_UR
     Output = {'allow_many_to_one': True,
                'allow_service_deterioration': True,
                'allow_urgent_relocation': True,
                'allow_vehicle_hiring': True,
                'batch_duration': '30',
                'clear_target_list_every_round': True,
                'contract_duration': '3600',
                'customer_segmentation': 'AA',
                'initial_fleet': '1000',
                'max_capacity': '06',
                'max_requests': '1000',
                'reinsert_targets': True,
                'time_horizon': '86400'}
    """
    label_setting_dic = instances_dic["labels"]
    
    # print(file_name)
        
    # E.g., ['BA-30', 'TH-86400', 'MR-1000', 'IF-1000', 'MC-06', 'CD-3600-SR-S1', 'CS-AA', 'SD', 'VH', 'MO', 'RT', 'CT', 'UR']
    file_instances = file_name.split("_")

    instance_settings = dict()

    for e in file_instances:
        
        if e in label_setting_dic.keys():
            # E.g., e =  SD
            k = label_setting_dic[e]
            # E.g., k = allow_service_deterioration
            instance_settings[k] = True
            
        else:
            # E.g., lv = ["BA", "30"]
            lv  = e.split('-')
            # E.g., e2 = BA
            e2 = lv[0]
            # E.g., k = batch_duration
            k2 = label_setting_dic[e2]
            
            if len(lv) > 1:
                 # E.g., v = '30'
                v = lv[1]
                instance_settings[k2] = v
            else:
                # label is not in instance name = False
                instance_settings[k] = False

    return instance_settings

### Aggregate results (folder round_track)

Instance fields in `round_track` folder. Every line is a snapshot of a simulation round of 30 seconds (first column is round `timestamp`):

#### Request status per round
* `waiting`
* `finished`
* `denied`
* `n_requests`

#### Freelance vehicles per round
* `hired_vehicles`
* `deactivated_vehicles`

#### Vehicle status per round
* `active_vehicles`
* `enroute_count`
* `parked_vehicles`
* `origin_vehicles`
* `simulation.rebalancing`
* `stopped_rebalancing`
* `idle`
* `picking_up`
* `O1,O2,O3,O4`
* `V1,V2,V3,V4`
* `distance_traveled_cruising`
* `distance_traveled_loaded`
* `distance_traveled_rebalancing`
* `run_time`

#### Vehicle status per round (seats)
* `seat_count`
* `picking_up_seats`
* `rebalancing_seats`
* `empty_seats`
* `total_capacity`

#### Service quality
* `pk_delay`
* `total_delay`
* `A_pk,A_dp,A_count,A_unmet_slevels`
* `B_pk,B_dp,B_count,B_unmet_slevels`
* `C_pk,C_dp,C_count,C_unmet_slevels`

In [4]:
import pandas as pd

def get_results_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}round_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
   

    df = pd.read_csv(experiment_file, index_col="timestamp",  parse_dates = True)

    # Number of requests
    total_requests = df["n_requests"].sum()
    serviced = df["finished"][-1]
    denied = df["denied"][-1]
    
    # Service quality
    sq_classes = ["A", "B", "C"]
    sq_settings = ["pk", "dp", "count", "unmet_slevels"]
    # e.g., ['A_pk', 'A_dp', 'A_count', 'A_unmet_slevels', 'B_pk', 'B_dp', 'B_count', 'B_unmet_slevels', 'C_pk', 'C_dp', 'C_count', 'C_unmet_slevels']
    sq_user_labels = ["{}_{}".format(sq_class, sq_setting) for sq_class in sq_classes for sq_setting in sq_settings]
    sq_user_dic = {l:df[l][-1] for l in sq_user_labels}
    
    # Distance traveled
    distance_cruising = df["distance_traveled_cruising"][-1]
    distance_loaded = df["distance_traveled_loaded"][-1]
    distance_rebalancing = df["distance_traveled_rebalancing"][-1]
    distance_total = distance_cruising + distance_loaded + distance_rebalancing

    # Pickup
    avg_pk_delay = df["pk_delay"].mean()
    avg_ride_delay = df["total_delay"].mean()

    # Separate occupancy labels (e.g., O1, O2, O3, etc.)
    occupancy_labels = [col for col in list(df) if 'O' in col]
    status_labels = [ "idle", "picking_up"] + occupancy_labels
    
    # Get fleet makeup
    fleet_makeup_labels = [col for col in list(df) if 'V' in col]
    fleet_makeup = {mk:df[mk][-1] for mk in fleet_makeup_labels}
    total_seats = {"seats_" + k:int(k[1:]) * v for k,v in fleet_makeup.items()}
    total_seats["total_seats"] = sum(total_seats.values())
    #max_hired = df["hired"].max()

    # Runtime
    total_runtime = df["run_time"].sum()
    
    # Dictionary of agreggate data
    dic_agreggate_data = {"serviced": "{:.2%}".format(serviced/total_requests),
                          "denied": "{:.2%}".format(denied/total_requests),
                          #'max_hired': max_hired,
                          "total_requests": total_requests,
                          "avg_pk_delay": "{:.2f}".format(avg_pk_delay),
                          "avg_ride_delay": "{:.2f}".format(avg_ride_delay),
                          "total_runtime": "{:.2f}".format(total_runtime/1000/60),
                          "distance_cruising": "{:.2%}".format(distance_cruising/distance_total),
                          "distance_loaded": "{:.2%}".format(distance_loaded/distance_total),
                          "distance_rebalancing": "{:.2%}".format(distance_rebalancing/distance_total),
                          "distance_total": distance_total//1000}
    
    # All data
    dic_ag = {**dic_agreggate_data, **fleet_makeup}
    dic_ag = {**dic_ag, **total_seats}
    dic_ag = {**dic_ag, **sq_user_dic}

    return dic_ag

### Aggregate results (folder request_track)

* `earliest`
* `id` = 1, 2, 3, ..., #USERS
* `class` = A, B, C
* `pk_delay`
* `ride_delay`
* `pk_time`
* `dp_time`
* `id_from` = Network id
* `id_to` = Network id
* `dist` = trip(id_from, id_to) in seconds
* `service` = {FLEET, FREELANCE}
* `service_level` = {MET, UNMET} (signals if min. service level was extended (UNMET), NOT whether a user was actually picked up under first-tier service levels. An extension might occur but a user can still be picked up within first-ter service levels due to hiring.

In [5]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    dfp = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay"])

    return dfp

In [6]:
import pandas as pd
from pprint import pprint

def get_request_track_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
    # print("Processing experiment file '{}'".format(experiment_file))
    df = pd.read_csv(experiment_file, index_col="earliest",  parse_dates = True)
    
    aggfunc = {"pk_delay" : ['mean', 'count', 'max']}
    
    # Also can be done with pivot table
    df_pivot = df.pivot_table(index="class", columns="service_level", aggfunc=aggfunc, values=["pk_delay", "id_from"])


    service_count = df['service'].value_counts().to_dict()
    class_count = df['class'].value_counts().to_dict()
    
    # How many users had their service levels fulfilled (MET)? How many had their service levels deteriorated, or have been rejected (UNMET)?
    service_level = df['service_level'].value_counts().to_dict()  
    
    # 
    service_quality_dic = {'A':{'MET': {'Avg.':None, 'Max.':None, 'Count':None}, 'UNMET': {'Avg.':None, 'Max.':None, 'Count':None}},
                           'B':{'MET': {'Avg.':None, 'Max.':None, 'Count':None}, 'UNMET': {'Avg.':None, 'Max.':None, 'Count':None}},
                           'C':{'MET': {'Avg.':None, 'Max.':None, 'Count':None}, 'UNMET': {'Avg.':None, 'Max.':None, 'Count':None}}}
 
    for sq_class in ['A', 'B', 'C']:
        for service_level in ["MET", "UNMET"]:
            filter_sq_sl = (df['class'] == sq_class) & (df['service_level'] == service_level)
            service_quality_dic[sq_class][service_level]['Max.'] = df.loc[filter_sq_sl]["pk_delay"].max()
            service_quality_dic[sq_class][service_level]['Avg.'] = df.loc[filter_sq_sl]["pk_delay"].mean()
            service_quality_dic[sq_class][service_level]['Count'] = df.loc[filter_sq_sl]["pk_delay"].count()
    
    # How many vehicles (freelance or from initial fleet) serviced each user (considering class and service quality status, that is, MET or UNMET)
    fleet_settings_dic = {sq_class:{service_level:df.loc[(df['class']==sq_class) & 
                                                         (df['service_level']==service_level)]["service"].value_counts().to_dict()
                                                   
                                    for service_level in  ["MET", "UNMET"]}
                          for sq_class in ['A', 'B', 'C']}
    
    dic_request_track = {"service_levels":service_quality_dic,
                          "fleet_settings":fleet_settings_dic}
    
    return dic_request_track

### Processing single instance

In [7]:
import pandas as pd

def get_aggregate_request_dic(path_experiment, name_experiment):
    
    # Load results
    experiment_file = "{}request_track/{}.csv".format(path_experiment, name_experiment)
    
   
    df = pd.read_csv(experiment_file, index_col = "earliest",  parse_dates = True)

    return df

### Processing all instances in folder

In [8]:
# Get all instances in folder
instance_file_names = os.listdir(request_log_folder)

# Only consider instances declared in the settings.json file
# instance_file_names = [i for i in instance_file_names if instances_dic['instance_name'] in i]

# Instance Info (key = instance name)
dic_all_instance_settings = dict()
dic_all_round_track = dict()
dic_all_request_track = dict()

print("Reading files in folder:", request_log_folder)
for file_name in instance_file_names:
    
    instance, extension = file_name.split(".")
    print("  - Processing", instance)
    
    # Instance settings
    instance_settings_dic = get_instance_settings(instance)
    round_track_agg_dic = get_results_dic(instances_folder, instance)
    request_track_agg_dic = get_request_track_dic(instances_folder, instance)
    
    # Get aggregated results
    dic_all_round_track[instance] = round_track_agg_dic
    dic_all_request_track[instance] = request_track_agg_dic
    dic_all_instance_settings[instance] = instance_settings_dic

Reading files in folder: C:/Users/LocalAdmin/IdeaProjects/slevels/instance_output/week_deny_ABCRT//request_track
  - Processing IN-PROFILETIME_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR
  - Processing IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR
  - Processing IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR
  - Processing IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-CC_RE_CT_RT_UR
  - Processing IN-WEEKDENYABCRT_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR
  - Processing IN-WEEKDENYABCRT_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR


C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\LocalAdmin\anaconda3\envs\env_slevels\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = metho

In [9]:
print("Instance settings:")
pprint(dic_all_instance_settings)

Instance settings:
{'IN-PROFILETIME_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR': {'allow_urgent_relocation': True,
                                                                         'batch_duration': '30',
                                                                         'clear_target_list_every_round': True,
                                                                         'customer_segmentation': 'BB',
                                                                         'initial_fleet': '1000',
                                                                         'instance_name': 'PROFILETIME',
                                                                         'max_capacity': '4',
                                                                         'max_requests': '1000',
                                                                         'rebalance': True,
                                                                         're

### Building indexes for multilevel table

In [10]:
from collections import defaultdict
import math

dict_sl_status = {"MET":"Met", "UNMET":"Unmet"}
filter_sl_status = ["MET"]
instance_attribute = defaultdict(list)
 
# Multilevel service level
for instance_name, dic_request_track in dic_all_request_track.items():
    
    instance_settings = get_instance_settings(instance_name)
    if instance_settings.get('service_rate', '-') == '-':
        continue
    
    column = ('', 'service_rate')
    instance_attribute[column].append(instance_settings['service_rate'])
    column = ('', 'contract_duration')
    instance_attribute[column].append(int(instance_settings['contract_duration']))
    column = ('', 'customer_segmentation')
    instance_attribute[column].append(instance_settings['customer_segmentation'])
    
    for sq_class, sl_status_dic in dic_request_track["service_levels"].items():
        for sl_status, attribute_dic in sl_status_dic.items():
            for attribute, value in attribute_dic.items():
                
                # Filter sl_status
                if sl_status in filter_sl_status:
                    continue
                
                column = (dict_sq_class[sq_class], attribute)
                if(column[1]=="Count"):
                    instance_attribute[column].append(value if not (math.isnan(value) or value==0) else "-")
                else:
                    instance_attribute[column].append(("{:.1f}".format(value/60) if not math.isnan(value) else "-"))


In [11]:
a = pd.DataFrame.from_dict(dic_all_round_track)
b = a.T
b

,serviced,denied,total_requests,avg_pk_delay,avg_ride_delay,total_runtime,distance_cruising,distance_loaded,distance_rebalancing,distance_total,...,A_count,A_unmet_slevels,B_pk,B_dp,B_count,B_unmet_slevels,C_pk,C_dp,C_count,C_unmet_slevels
IN-PROFILETIME_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR,89.98%,10.02%,4072,173.10,229.91,2.42,12.13%,47.34%,40.53%,22254,...,378,336,193.85,274.3,2685,72,302.82,396.25,601,0
IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR,64.22%,35.78%,4072,149.43,188.03,3.41,11.60%,35.98%,52.42%,21979,...,1374,1448,205.58,279.61,640,9,248.36,387.03,601,0
IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR,89.02%,10.98%,4072,173.88,235.26,2.65,12.38%,46.38%,41.23%,22384,...,363,351,194.84,279.51,2661,96,305.02,408.64,601,0
IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-CC_RE_CT_RT_UR,91.85%,8.15%,4072,208.67,260.97,2.28,14.79%,47.16%,38.05%,22736,...,396,318,189.39,267.68,635,14,286.69,360.37,2709,0
IN-WEEKDENYABCRT_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR,nan%,nan%,0,nan,nan,0.02,nan%,nan%,nan%,0,...,0,0,NaN,NaN,0,0,NaN,NaN,0,0
IN-WEEKDENYABCRT_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR,nan%,nan%,0,nan,nan,0.04,nan%,nan%,nan%,0,...,0,0,NaN,NaN,0,0,NaN,NaN,0,0


## Isolate instance settings entries

In [12]:
df_instance_settings = pd.DataFrame.from_dict(dic_all_instance_settings)
df_instance_settings = df_instance_settings.T
df_instance_settings = df_instance_settings[["customer_segmentation", "instance_name"]]
df_instance_settings["exp_label"] = df_instance_settings.apply(lambda row: f'{row["customer_segmentation"]}_{row["instance_name"]}', axis=1)
df_instance_settings

,customer_segmentation,instance_name,exp_label
IN-PROFILETIME_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR,BB,PROFILETIME,BB_PROFILETIME
IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR,AA,WEEKDENYABCRT,AA_WEEKDENYABCRT
IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR,BB,WEEKDENYABCRT,BB_WEEKDENYABCRT
IN-WEEKDENYABCRT_BA-30_ST-3600_MR-1000_IF-1000_MC-4_CS-CC_RE_CT_RT_UR,CC,WEEKDENYABCRT,CC_WEEKDENYABCRT
IN-WEEKDENYABCRT_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-AA_RE_CT_RT_UR,AA,WEEKDENYABCRT,AA_WEEKDENYABCRT
IN-WEEKDENYABCRT_BA-30_ST-604800_MR-1000_IF-1000_MC-4_CS-BB_RE_CT_RT_UR,BB,WEEKDENYABCRT,BB_WEEKDENYABCRT


In [13]:
df_compare = pd.concat([df_instance_settings, b], axis=1)
df_compare.to_csv(f"{label}.csv")

In [14]:
df_compare_summary = df_compare[["exp_label", "customer_segmentation", "A_unmet_slevels", "B_unmet_slevels", "C_unmet_slevels", "distance_total","serviced"]]
df_compare_summary = df_compare_summary.sort_values(by=["customer_segmentation", "serviced"], ascending=[True, False])
df_compare_summary = df_compare_summary.set_index(["exp_label"])
df_compare_summary.to_csv(f"summary_{label}.csv")
df_compare_summary

,customer_segmentation,A_unmet_slevels,B_unmet_slevels,C_unmet_slevels,distance_total,serviced
exp_label,,,,,,
AA_WEEKDENYABCRT,AA,0,0,0,0,nan%
AA_WEEKDENYABCRT,AA,1448,9,0,21979,64.22%
BB_WEEKDENYABCRT,BB,0,0,0,0,nan%
BB_PROFILETIME,BB,336,72,0,22254,89.98%
BB_WEEKDENYABCRT,BB,351,96,0,22384,89.02%
CC_WEEKDENYABCRT,CC,318,14,0,22736,91.85%


### Table: What is the service level (pickup delay) of the users lying outside SQ-class service rate?

In [15]:
a = pd.DataFrame.from_dict(instance_attribute)

# Establishing category order and alias dictionaries
dict_segmentation = {"AA":"B+", "BB":"S+", "CC":"L+", "A":"B", "B":"S", "C":"L"}
category_segmentation = pd.api.types.CategoricalDtype(categories=["B+", "S+", "L+", "B", "S", "L"], ordered=True)

dict_contract_duration = {3600:"1h", 0:"Single-ride", 10800:"3h"}
category_contract_duration = pd.api.types.CategoricalDtype(categories=["Single-ride", "1h", "3h"], ordered=True)

dict_service_rate = {"S1":"SR1", "S2":"SR2", "S3":"SR3"}
category_service_rate = pd.api.types.CategoricalDtype(categories=["SR1", "SR2", "SR3"], ordered=True)

key_sr = ('', 'service_rate') # (S1, S2, S3)
key_cs = ('', 'customer_segmentation') # (A, AA, BB, etc.)
key_cd = ('', 'contract_duration') #(0, 3600, 18000)

# Filtering data
a = a[a[key_cs].isin(["AA",
                      "BB",
                      "CC"])]

# Renaming data and applying aliases
a[key_cs] = a[key_cs].map(lambda e:dict_segmentation[e])
a[key_cs] = a[key_cs].astype(category_segmentation)

a[key_cd] = a[key_cd].map(lambda e:dict_contract_duration[e])
a[key_cd] = a[key_cd].astype(category_contract_duration)

a[key_sr] = a[key_sr].map(lambda e:dict_service_rate[e])
a[key_sr] = a[key_sr].astype(category_service_rate)

a = a.sort_values(by=[key_sr, key_cs, key_cd])
a = a.set_index([key_sr, key_cs, key_cd])

a.index.names = ["Service rate",
                 "Segmentation scenario",
                 "Contract duration"]

a

# Getting latex table
# print(a.to_latex(multicolumn=True, multirow=True))

KeyError: ('', 'customer_segmentation')